# Verizon Spotcheck
We re-collected Boston addresses and looked at differences in cost and speed.

In [1]:
import glob
from tqdm import tqdm
from multiprocess import Pool

import pandas as pd

from parsers import verizon_workflow

In [2]:
# inputs
fn_verizon = '../data/output/speed_price_verizon.csv.gz'
pattern_spotcheck = '../data/intermediary/isp_spotcheck/verizon/*/*.geojson.gz'

In [3]:
data_verizon = []
files = glob.glob(pattern_spotcheck)
with Pool(20) as pool:
    for record in tqdm(pool.imap_unordered(verizon_workflow, files), 
                       total=len(files)):
        data_verizon.extend(record)
verizon_spot = pd.DataFrame(data_verizon)
del data_verizon

100%|██████████| 549/549 [00:01<00:00, 433.51it/s]


In [4]:
verizon = pd.read_csv(fn_verizon, compression='gzip')

/home/chino/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
verizon.head(2)

,address_full,incorporated_place,major_city,state,lat,lon,availability_status,block_group_x,collection_datetime,in_service,...,lon_closest_fiber,block_group_y,geoid,race_perc_non_white,income_lmi,ppl_per_sq_mile,n_providers,income_dollars_below_median,internet_perc_broadband,median_household_income
0,"1305 QUINCY ST NW, WASHINGTON, DC, 20011-5525,...",Washington city,washington,DC,38.937824,-77.030087,NaN,110010025023,1650485042,True,...,-77.030087,3,110010025023,0.603035,1.522468,8304.094343,3.0,-47462.0,1.0,138304
1,"1342 QUINCY ST NW, WASHINGTON, DC, 20011-5526,...",Washington city,washington,DC,38.937413,-77.031098,NaN,110010025023,1650485004,True,...,-77.031098,3,110010025023,0.603035,1.522468,8304.094343,3.0,-47462.0,1.0,138304


In [6]:
cols = [
    'price_new', 'price_old', 'speed_down_new', 'speed_down_old'
]

In [7]:
verizon_spot = verizon_spot.merge(verizon[['lat', 'lon', 'price', 'speed_down', 'race_perc_non_white', 'income_dollars_below_median']],
                                  how='inner',
                                  on=['lat', 'lon'], suffixes=["_new", "_old"])

In [8]:
verizon_spot[verizon_spot['price_new'] != verizon_spot['price_old']][cols]

,price_new,price_old,speed_down_new,speed_down_old
0,NaN,40.00,0.0,10.0
1,NaN,NaN,0.0,0.0
2,NaN,NaN,0.0,0.0
3,NaN,NaN,0.0,0.0
4,NaN,NaN,0.0,0.0
...,...,...,...,...
13428,NaN,39.99,0.0,300.0
13431,49.99,39.99,300.0,300.0
13434,39.99,49.99,300.0,300.0
13439,39.99,49.99,300.0,300.0


In [21]:
verizon_spot.price_new.replace({40: 39.99}, inplace=True)
verizon_spot.price_old.replace({40: 39.99}, inplace=True)

In [9]:
verizon_spot.price_new.describe(), verizon_spot.price_old.describe()

(count    9838.000000
 mean       40.507827
 std         2.514814
 min        39.990000
 25%        39.990000
 50%        39.990000
 75%        39.990000
 max        84.990000
 Name: price_new, dtype: float64,
 count    9823.000000
 mean       42.345551
 std         4.389192
 min        39.990000
 25%        39.990000
 50%        39.990000
 75%        40.000000
 max        64.990000
 Name: price_old, dtype: float64)

In [23]:
price_changed = verizon_spot[(verizon_spot['price_new'] != verizon_spot['price_old']) &
             (~verizon_spot.price_new.isnull()) &
             (~verizon_spot.price_old.isnull())]

In [11]:
price_changed[cols]

,price_new,price_old,speed_down_new,speed_down_old
15,49.99,39.99,300.0,300.0
25,39.99,49.99,300.0,300.0
27,49.99,39.99,300.0,300.0
28,39.99,49.99,300.0,300.0
34,39.99,49.99,300.0,300.0
...,...,...,...,...
13427,39.99,49.99,300.0,300.0
13431,49.99,39.99,300.0,300.0
13434,39.99,49.99,300.0,300.0
13439,39.99,49.99,300.0,300.0


In [24]:
len(price_changed[
    price_changed['price_new'] > price_changed['price_old']
]) / len(price_changed)

0.12774957698815567

In [25]:
len(price_changed[
    price_changed['price_new'] < price_changed['price_old']
]) / len(price_changed)

0.8722504230118443

In [26]:
# note none of the speeds change.
speed_changed = verizon_spot[
    (verizon_spot['speed_down_new'] != verizon_spot['speed_down_old']) &
    (~verizon_spot['price_new'].isnull()) &
    (~verizon_spot['price_old'].isnull())
]

In [46]:
len(verizon_spot[verizon_spot.speed_down_old <= 25])

4030

In [44]:
len(speed_changed)

340

In [27]:
speed_changed[speed_changed['speed_down_new'] == 300][cols]

,price_new,price_old,speed_down_new,speed_down_old
5498,39.99,39.99,300.0,10.0


In [28]:
speed_changed[speed_changed['speed_down_old'] == 300][cols]

,price_new,price_old,speed_down_new,speed_down_old
1147,39.99,39.99,50.0,300.0


In [47]:
speed_changed['speed_down_old'].value_counts()

10.0     329
7.1        9
300.0      1
15.0       1
Name: speed_down_old, dtype: int64

all but one instance of a household being offered different speeds was for Verizon's "High Speed Internet"

In [29]:
verizon_spot[verizon_spot['price_new'] > 39.99].income_dollars_below_median.describe()

count       441.000000
mean       3131.877551
std       46188.593798
min     -108990.000000
25%      -32720.000000
50%       13459.000000
75%       47704.000000
max       67113.000000
Name: income_dollars_below_median, dtype: float64

In [30]:
verizon_spot[verizon_spot['price_old'] > 39.99].income_dollars_below_median.describe()

count      2151.000000
mean      -7185.992562
std       46976.896763
min     -119952.000000
25%      -49483.000000
50%       -6410.000000
75%       35794.500000
max       67113.000000
Name: income_dollars_below_median, dtype: float64

In [31]:
verizon_spot[verizon_spot['price_new'] > 39.99].race_perc_non_white.describe()

count    456.000000
mean       0.514393
std        0.310580
min        0.000000
25%        0.237150
50%        0.521895
75%        0.831577
max        1.000000
Name: race_perc_non_white, dtype: float64

In [32]:
verizon_spot[verizon_spot['price_old'] > 39.99].race_perc_non_white.describe()

count    2264.000000
mean        0.467853
std         0.322709
min         0.000000
25%         0.175947
50%         0.447368
75%         0.762159
max         1.000000
Name: race_perc_non_white, dtype: float64

In [33]:
# median income for everything
verizon_spot.income_dollars_below_median.describe()

count     12534.000000
mean      -2769.074358
std       46325.610972
min     -138247.000000
25%      -39119.000000
50%        2965.000000
75%       39965.000000
max       67113.000000
Name: income_dollars_below_median, dtype: float64

In [34]:
verizon_spot.race_perc_non_white.describe()

count    13452.000000
mean         0.491652
std          0.313279
min          0.000000
25%          0.220965
50%          0.447368
75%          0.822259
max          1.000000
Name: race_perc_non_white, dtype: float64

The median income was lower in higher cost plans from the first sample, but not the second sample.

The racial demographics were less white for more expensive plans

In [37]:
verizon_spot[verizon_spot.price_old != '39.99'].price_new.value_counts(normalize=True)

39.99    0.953649
49.99    0.042895
64.99    0.003354
84.99    0.000102
Name: price_new, dtype: float64

In [38]:
verizon_spot[verizon_spot.price_new != '39.99'].price_old.value_counts(normalize=True)

39.99    0.769521
49.99    0.227120
64.99    0.003359
Name: price_old, dtype: float64

95 percent of offers that were over 39.99 were switched back to 39.99 when we checked again. Similarly, 77 percent of offers that are now over 39.99 were originally 39.99.